<a href="https://colab.research.google.com/github/thesandi99/Open-ARC/blob/master/%5BT_1%5D_Open_ARC_Model_Traning_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/thesandi99/Open-ARC.git
%cd Open-ARC

Cloning into 'Open-ARC'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 118 (delta 50), reused 85 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 71.95 KiB | 2.77 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Open-ARC


In [ ]:
import json
from openarc.model import OpenTokenizer, OpenARC
import torch

task_json_data = '{"train":[{"input":[[0,0,4,0,0,0,0,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,4,0,0,0,0,0,0,0,0,0]],"output":[[0,0,0,0,0,0,0,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]},{"input":[[0,0,0,2,0,0,2,0,0,2,0,0,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0]],"output":[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0]]},{"input":[[4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,4,0,0,0,0,4,0,0,0,0,0,0,0,0,0]],"output":[[4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]}],"test":[{"input":[[3,3,3,3,3,3,3,3,3,3,3,3,0,0,3,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0]]}]}'
solution = '{"1d_denoising_1c_0":[[3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]}'
solution_ = json.loads(solution)
task_json_data = json.loads(task_json_data)

tokenizer = OpenTokenizer.from_pretrained()
prompt = tokenizer.tokenize_task(task_json_data=task_json_data, true_label_data=solution_, task_id="1d_denoising_1c_0", max_seq_len=8192)
# print(prompt)
prompt = prompt['prompt_tensor'].clone().detach().to("cuda:0")
model = OpenARC().to('cuda:0')
output, _ = model(prompt)
output = torch.argmax(output, dim=-1)
print(output)

tensor([[4, 4, 4, 4, 4, 7, 4, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4,
         7, 4, 4, 7, 7, 7, 7, 7, 4, 4, 7, 7, 7, 1, 4, 4, 4, 4, 7, 7, 4, 7, 7, 7,
         4, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 4, 4, 7, 7, 4, 4, 4,
         7, 7, 4, 1, 4, 7, 4, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 4, 4, 9, 7, 4, 7, 4,
         4, 9, 4, 7, 4, 4, 9, 4, 4, 4, 7, 4, 4, 4, 7, 1, 1, 4, 4, 4, 4, 7, 7, 7,
         4, 7, 7, 4, 4, 7, 7, 4, 7, 4, 4, 4, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7,
         9, 7, 7, 4, 4, 4, 7, 9, 4, 4, 7, 7, 4, 4, 7, 7, 4, 4, 4, 7, 7, 7, 7, 7,
         4, 4, 7, 7, 7, 4, 7, 7, 7, 7, 7, 4, 7, 4, 7, 4, 4, 4, 4, 4, 7, 4, 7, 4,
         4, 7, 7, 7, 7, 4, 4, 7, 4, 7, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7,
         7, 4, 4, 4, 4, 4, 1, 1, 7, 4, 4, 4, 7, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4,
         4, 7, 4, 4, 4, 7, 4, 4, 7, 7, 4, 4, 7, 4, 7, 7, 7, 4, 7, 4, 7, 1, 4]],
       device='cuda:0')


In [ ]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import torch

dummy_train_labels = torch.tensor(solution_['1d_denoising_1c_0'])

criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Starting simple training...")
model.train() # Set the model to training mode
device = next(model.parameters()).device
print(f"Model is on device: {device}")

for epoch in range(50):  # Train for a single epoch for demonstration
    optimizer.zero_grad()
    prompt = prompt.to(device)
    train_output, _ = model(prompt)
    print(f"Shape of model output (train_output): {train_output.shape}")
    print(f"Shape of target labels (dummy_train_labels): {dummy_train_labels.shape}")

    required_seq_len = train_output.shape[1]
    if dummy_train_labels.shape[1] != required_seq_len:
        print(f"Warning: Target label sequence length ({dummy_train_labels.shape[1]}) does not match model output sequence length ({required_seq_len}).")
        if dummy_train_labels.shape[1] < required_seq_len:
             padding_value = -100

             # Pad the dummy_train_labels tensor
             padded_labels = torch.full((dummy_train_labels.shape[0], required_seq_len), padding_value, dtype=dummy_train_labels.dtype)
             padded_labels[:, :dummy_train_labels.shape[1]] = dummy_train_labels
             aligned_train_labels = padded_labels
             print(f"Padded dummy_train_labels to shape: {aligned_train_labels.shape}")
        else: # dummy_train_labels.shape[1] > required_seq_le
             aligned_train_labels = dummy_train_labels[:, :required_seq_len]
             print(f"Cropped dummy_train_labels to shape: {aligned_train_labels.shape}")
    else:
        aligned_train_labels = dummy_train_labels
        print(f"Target labels already match model output sequence length.")

    aligned_train_labels = aligned_train_labels.to(device)
    print(f"Aligned target labels moved to device: {aligned_train_labels.device}")

    train_output_permuted = train_output.permute(0, 2, 1) # Change shape from [1, 263, 20] to [1, 20, 263]
    print(f"Shape of permuted model output: {train_output_permuted.shape}")

    loss = criterion(train_output_permuted, aligned_train_labels) # Use permuted output

    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

print("Training finished.")

print("\nStarting simple evaluation...")
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Disable gradient calculation for evaluation
    prompt = prompt.to(device)
    eval_output, _ = model(prompt)
    predicted_output = torch.argmax(eval_output, dim=-1)
    true_label_tensor = torch.tensor(solution_['1d_denoising_1c_0']).squeeze(0).to(device) # Adjust based on your data structure and move to device
    print(f"Shape of predicted output (argmax): {predicted_output.shape}")
    print(f"Shape of true label for evaluation: {true_label_tensor.shape}")

    min_len = min(predicted_output.shape[-1], true_label_tensor.shape[-1])
    predicted_compare = predicted_output[0, :min_len] # Assuming batch size 1
    true_label_compare = true_label_tensor[:min_len]

    print("Predicted Output (Comparison Slice):")
    print(predicted_compare)
    print("True Label (Comparison Slice):")
    print(true_label_compare)

    correct_predictions = (predicted_compare == true_label_compare).sum().item()
    total_elements = min_len
    accuracy = correct_predictions / total_elements
    print(f"Simple Accuracy (comparing first {min_len} tokens): {accuracy:.4f}")


print("Evaluation finished.")

Starting simple training...
Model is on device: cuda:0
Shape of model output (train_output): torch.Size([1, 263, 20])
Shape of target labels (dummy_train_labels): torch.Size([1, 32])
Padded dummy_train_labels to shape: torch.Size([1, 263])
Aligned target labels moved to device: cuda:0
Shape of permuted model output: torch.Size([1, 20, 263])
Epoch 1, Loss: 3.388619899749756
Shape of model output (train_output): torch.Size([1, 263, 20])
Shape of target labels (dummy_train_labels): torch.Size([1, 32])
Padded dummy_train_labels to shape: torch.Size([1, 263])
Aligned target labels moved to device: cuda:0
Shape of permuted model output: torch.Size([1, 20, 263])
Epoch 2, Loss: 1.0043022632598877
Shape of model output (train_output): torch.Size([1, 263, 20])
Shape of target labels (dummy_train_labels): torch.Size([1, 32])
Padded dummy_train_labels to shape: torch.Size([1, 263])
Aligned target labels moved to device: cuda:0
Shape of permuted model output: torch.Size([1, 20, 263])
Epoch 3, Loss: